In [1]:
import re
import math
import sys
import random

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity

from data_tokenizer import load_corpus

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 269
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1313 words
filter frequent words: 540 words
filter letter words: 539 words
filter stop words: 352 words


In [4]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### SDC

In [6]:
def get_bow(corpus):
    new_dict = Dictionary(corpus)

    # new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))
    # print(len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)
    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
            array[i] = numpy.divide(array[i], len(idx_corpus[i]))

    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [7]:
def sdc(bow_corpus, min_samples, eps):
    delta_eps = eps / 10
    labels = [-1 for i in range(len(bow_corpus))]
    sims = cosine_similarity(bow_corpus)
    
    points = [i for i in range(len(bow_corpus))]
    cluster_num = 0
    while len(points) > 0:
        seed = random.choice(points)
        eps_neighbors = [i for i, sim in enumerate(sims[seed]) if sim >= eps and labels[i] == -1]
        if len(eps_neighbors) >= min_samples:
            cluster_num += 1
            for p in eps_neighbors:
                labels[p] = cluster_num
            points = [i for i in points if i not in eps_neighbors]
        else:
            labels[seed] = 0
            points.remove(seed)

    while cluster_num != 0:
        cluster = [numpy.array(bow_corpus.iloc[i]) for i, label in enumerate(labels) if label == cluster_num]
        eps_temp = eps
        
        while True:
            centroid = numpy.mean(cluster, axis=0).reshape(1, -1)
            eps_temp -= delta_eps
            
            count = 0
            for i, label in enumerate(labels):
                point = numpy.array(bow_corpus.iloc[i]).reshape(1, -1)
                if label == 0 and cosine_similarity(centroid, point) >= eps_temp:
                    cluster.append(point[0])
                    labels[i] = cluster_num
                    count += 1
            if count == 0:
                break
        
        cluster_num -= 1
    
    return labels

In [27]:
def upgrade_sdc(bow_corpus, min_samples, eps):
    delta_eps = eps / 20
    labels = [-1 for i in range(len(bow_corpus))]
    initials = []
    
    clusters = []
    clusters.append([])
    cluster_num = 0
    
    points = [i for i in range(len(bow_corpus))]
    sims = cosine_similarity(bow_corpus)
    while len(points) > 0:
        seed = random.choice(points)
        eps_neighbors = [i for i, sim in enumerate(sims[seed]) if sim >= eps and labels[i] == -1]
        if len(eps_neighbors) >= min_samples:
            print(seed)
            cluster_num += 1
            clusters.append([])
            for p in eps_neighbors:
                initials.append(p)
                labels[p] = cluster_num
                clusters[cluster_num].append(numpy.array(bow_corpus.iloc[p]))
            points = [i for i in points if i not in eps_neighbors]
        else:
            labels[seed] = 0
            clusters[0].append((seed, numpy.array(bow_corpus.iloc[seed])))
            points.remove(seed)
    
    expandable = numpy.zeros(cluster_num + 1)    
    while numpy.sum(expandable) != -cluster_num - 1:
        eps -= delta_eps
        count = numpy.zeros(cluster_num + 1)
        for point in clusters[0]:
            if labels[point[0]] != 0:
                continue

            num = point[0]
            p = point[1].reshape(1, -1)
            max_sim = 0
            for c, cluster in enumerate(clusters[1:]):
                if expandable[c + 1] == -1:
                    continue
                centroid = numpy.mean(cluster, axis=0).reshape(1, -1)
                if cosine_similarity(centroid, p) >= max_sim:
                    max_sim = cosine_similarity(centroid, p)
                    if max_sim >= eps:
                        labels[num] = c + 1

            if labels[num] != 0:
                count[labels[num]] += 1
                clusters[labels[num]].append(point[1])

        for i, num in enumerate(count):
            if num == 0:
                expandable[i] = -1
        
    return labels, initials

In [48]:
def generate_result(predicted_labels):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = filtered_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    return result

In [49]:
min_samples = 7
eps = 0.3

# bow_corpus = get_bow(idx_corpus)
bow_corpus = get_bow(filtered_corpus)
predicted_labels, initials = upgrade_sdc(bow_corpus, min_samples, eps)
result = generate_result(predicted_labels)
bow_corpus.head()

161
170
8
70
66
97
155
114
14
185
177
145
213
105


,ทำ,พนักงาน,ข้าว,ดี,น้ำ,อี,เครื่อง,เลิก,กก,งง,...,พลังงาน,คอย,แอร์ไม่,สลิป,คู,ปอง,นวมินทร์,สิบ,ห้า,แจก
0,0.107143,0.214286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.083333,0.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.100000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Result

In [50]:
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
num_cluster = label_count[0][-1] + 1
print(label_count)

clusters = list(set(labels))
for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(len(clusters)):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]), array([46, 11, 12, 54, 16, 15, 18, 27, 13, 11, 12, 10,  9,  7,  8]))
		percent
0  |	46	1.0


In [51]:
clusters = [[] for i in range(num_cluster)]
corpus_centroid = []
for i, label in result['predicted_label'].iteritems():
    clusters[label].append(numpy.array(bow_corpus.iloc[i]))
    corpus_centroid.append(numpy.array(bow_corpus.iloc[i]))
corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
summation = 0
centroids = []
for i in range(num_cluster):
    size = len(clusters[i])
    if size != 0:
        print(i, end='\t')
        centroid = numpy.mean(clusters[i], axis=0)
        centroids.append(centroid)
        centroid = centroid.reshape(1, -1)
        similarities = cosine_similarity(centroid, clusters[i])
        pairwises = cosine_similarity(clusters[i])
        intra = numpy.sum(similarities) / size
        inter = cosine_similarity(centroid, corpus_centroid)[0][0]
        print(intra, end='\t')
        print(inter, end='\t')
        print(intra / inter)
        summation += numpy.sum(similarities)
    print()
print('Summation', summation)

	Intra cluster sim	Inter cluster sim	Intra / Inter
0	0.18239378143216803	0.36753180972699795	0.4962666539466335

1	0.6962331608556978	0.2718383587133487	2.561202782973943

2	0.586424372094304	0.20340885588735638	2.8829834843525872

3	0.5369440497120879	0.7728185798172112	0.6947866727519505

4	0.633032626079052	0.7103623685103349	0.8911404293650195

5	0.5656760352873816	0.3411491621633748	1.658148686926811

6	0.6540444066886858	0.43109246697722675	1.517178927469491

7	0.5137713746672198	0.5033594835994785	1.0206848016318015

8	0.6239335079908249	0.4366437776742164	1.4289302628202043

9	0.58701275248898	0.223678868662395	2.624354978185156

10	0.6107036386099756	0.593447668635797	1.0290774922308585

11	0.5901365569672661	0.21389050522915012	2.7590591566232794

12	0.5547875329246003	0.4512725761943406	1.2293845498062814

13	0.5760919637352335	0.43140815871327837	1.3353756810105082

14	0.5496688407686547	0.44590553263912797	1.232702445998809

Summation 137.56020623073277


In [75]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            new_labels = [new_labels[j] if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len_corpus)
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
14 3 0.47916215943434615
13 5 0.3641780584496489
13 3 0.32462701244897446
12 9 0.31011426594198627
12 4 0.3267286790372865
12 3 0.3262638475920433
10 4 0.7168296556743063
10 3 0.5987321008560234
8 6 0.9053426323056791
7 3 0.2889755478713765
4 3 0.7642711619246361
[0, 1, 2, 3, 3, 3, 6, 3, 6, 3, 3, 11, 3, 3, 3]


In [76]:
label_count = numpy.unique(new_result['predicted_label']) 

grouped_widget = widgets.ToggleButtons(
    options=[int(num) for num in label_count],
    disabled=False,
    button_style='',
)

def on_grouped_widget_click(change):
    clear_output()
    display(grouped_widget)
    change['new'] = grouped_widget.options[change['new']]
    for index, value in new_result[new_result['predicted_label'] == change['new']]['comment'].iteritems():
        if index in initials:
            print("*", end="")
        print(index, value)

grouped_widget.observe(on_grouped_widget_click, names='index')
on_grouped_widget_click({'new' : 0})

ToggleButtons(index=3, options=(0, 1, 2, 3, 6, 11), value=3)

*0 บางครั้งพนักงานโลตัสเอ็กเพลส กับพนักงานโลตัสใหญ่ ก็ต่างกันครับ โลตัสใหญ่ เขาทำเป็นเเผนกๆไปครับ เเต่โลตัสเอ็กเพลสนั้น พนักงานเเต่ละคน ต้องทำทุกอย่างครับ ไม่ใช่เเค่ยืนขายของ อยากให้ทุกคนเข้าใจด้วยครับ บางคนเวลากินข้าวเเทบไม่มี ผมเชื่อว่าทุกคนก็เป็นพนักงานเงินเดือนเหมือนกัน ควรเข้าใจกันบ้างครับ พนักงานคนอื่นเป็นเเบบไหนผมไม่รู้ เเต่มันก็ยังมีพนักงานที่ตั้งใจทำงานครับ ขอให้ทุกคนเข้าใจครับ
*4 เคยไปทำ เอ้กซ์เพรส เดือนเดียว พอเลย  พนักงานต่อ1ช่วงกะงานน้อยไป  3คน  บางวัน2 คน  คนเข้าวันนึงเกือบพัน ยิ่งช่วงเช้าเย็นเร่งด่วน วุ่นวายจัด
*6 โลตัสใหญ่ไม่มีอะไรนะ โลตัสเอกเพรสแม่งไม่ไหวจริงๆ เกลียดขี้หน้าพนักงาน
*7 ทุกคนล้วนแล้ว แต่มีหน้าที่ของเขา คนเราควรให้โอกาสซึ่งกันและกันนะครับให้คิดว่าคนดีก็มี คนไม่ดีก็มีนะครับ ผมไม่ได้โลกสวยนะครับ แต่ผมก็ทำงานในด้านบริการเหมือนกัน ถ้าไม่มายืนตรงจุดที่เขายืนคุณก็จะไม่รู้ คิดบวกนะครับ พนักงานเหมือนกันนะครับ ปรับปรุงตรงไหนแล้วดี ก็ควรที่ต้องปรับปรุงแก้ไขส่วนที่ผิดพลาดของตัวเรา ดีแล้วยังดีได้อีกนะครับ
*8 พนักงานทำหน้าเป็นตูดเกือบทุกคน ประมาณว่าเหมือนถูกบังคับให้มา

In [59]:
comment_widget = widgets.ToggleButtons(
    options=[num for num in range(num_cluster)],
    disabled=False,
    button_style='',
)

def on_comment_widget_click(change):
    clear_output()
    display(comment_widget)
    for index, value in result[result['predicted_label'] == change['new']]['comment'].iteritems():
        if index in initials:
            print("*", end="")
        print(index, value)

comment_widget.observe(on_comment_widget_click, names='index')
on_comment_widget_click({'new' : 0})

ToggleButtons(index=14, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14), value=14)

*20 เช็คเอาท์  หน้าจะงอไปไหนสวัสดีค่ะมีอะไรให้ช่วยมั้ยคะ
*23 ควย ค่ะ พนักงานแม่งเกือบทุกคนเลย ไม่มีความคิด วิเคราะห์ แยกแยะไม่ได้ว่าเวลาลูกค้าซื้อผัก ซื้อหมูบด น้ำยาล้างจาน น้ำปลา หรืออะไรก็แล้วแต่ ถุงมึงมีใบใหญ่สุดแค่ไหนมึงเอามาใส่แม่งรวมกันหมดทุกอย่าง ผักกูเลือกดีๆเอาไม่ช้ำ อีดอกเอาไว้ล่างสุด เอาแพ็คหมูบดทับ
*72 พนักงานสาขาย่อย เขาเรียกเอกซ์เพรสปะ หน้าบูดกวนตีน ถามอะไรไม่ตอบ เป็นหบายสาขาเลย เคยยืนรอให้มันคิดตังให้ ต้องเรียกอะ ขนาดวางของที่เค้าเตอร์แล้ว มัวยืนคุยกับเพื่อน ทั้งๆที่มันก็เห็น
*105 คือแคชเชียมึงจะรีบไปไหนชอบไล่กุไปรอด้านหน้าทั้งที่ของกุยังไม่ได้คิดตังแถมรีบทอนตังตาลีตาเหลือกยัดตังทอนใส่มือไม่มองหน้ากุสักนิด พอไปตรงเค้าเตอร์ประชาสัมพันพนงหน้าบอกบุญไม่รับไม่สนใจลูกค้าอีกตะหาก นี่ตกลงกุมาซื้อหรือมาขอฟรีๆกันแน่
*120 พนักงานอย่าคุยเล่น หรือด่ากัน เสียงดัง และหยาบ ต่อหน้าลูกค้า
*196 เอ็กเพลส พนง เกือบทุกสาขา หน้าไม่รับแขก
*203 โลตัสเล็กพนักงานหน้าหงิกเป็นส้นตีนเลยจ้า
252 เอกเพรส ซทานสัมฤทธิ์ หน้าหงิกเป็นเล็บขบ


In [37]:
token_widget = widgets.ToggleButtons(
    options=[num for num in range(num_cluster)],
    disabled=False,
    button_style='',
)

def on_token_widget_click(change):
    clear_output()
    display(token_widget)
    for index, value in result[result['predicted_label'] == change['new']]['tokenized_comment'].iteritems():
        print(index, value)

token_widget.observe(on_token_widget_click, names='index')
on_token_widget_click({'new' : 0})

ToggleButtons(index=11, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=11)

2 ['ถุง', 'ถุง', 'เจอ', 'น้ำ', 'งง', 'น้ำยา', 'ผ้า', 'ผ้า', 'กก']
3 ['ถุง', 'ถุง', 'ลด', 'แถม', 'ดู', 'หนา', 'ลอง']
97 ['ลูกค้า', 'ถุง', 'ใส่', 'เรื่อง', 'ซ้อน', 'อบรม', 'กิน', 'กิน', 'อย่า']
114 ['ถุง', 'ถุง', 'ขนาด', 'หนา', 'ชื้อ', 'ลูก']
157 ['ถุง', 'เวลา', 'บ้าน', 'หนา', 'ชื่อ', 'แถบ', 'แถบ']
173 ['ถุง', 'ถุง', 'ซ้อน', 'กิน', 'แม่ง', 'หรอ']
187 ['ถุง', 'ถุง', 'ดี', 'ลด', 'เลิก', 'ขาย', 'พลาสติก', 'แทน', 'โลก']
189 ['ถุง', 'ก้อ', 'หี']
195 ['ถุง', 'ถุง', 'โดน', 'น้ำยา', 'ผ้า', 'นุ่ม']
204 ['โลตัส', 'ถุง', 'แรง', 'เกลียด']
217 ['ถุง', 'ใบ', 'ซ้อน', 'ขนาด']
258 ['ถุง', 'เลิก', 'พลาสติก', 'แจก']
264 ['ถุง', 'ชอบ', 'เน่า']


In [ ]:
seed = 244
compare = 89

a = numpy.array(bow_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(bow_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))
print(filtered_corpus[seed])
print(filtered_corpus[compare])